In [26]:
# Audit Graph

#------------
# Imports and Classes
#------------
import graphviz
import sqlparse
import pandas as pd
import re
import os

class MyStmts:
    def __init__(self):
        self.tokens = []
        self.ttype = None
        self.value = ''

#------------
# Helper 01 (used in Main 01)
#------------
def gen_pl(query):
    '''
    pl: parsed list
    '''

    # initialize df_q
    df_q = pd.DataFrame()
    df_q.at[0, 'QUERY_TEXT'] = query # assign sql text to df cell
    
    # initialize pl, to handle multiple statements
    pl = MyStmts()

    # process pl
    for index, row in df_q.iterrows(): #multiple queries in SF
        sql = row['QUERY_TEXT']
        sql = re.sub(r'--.*', '', sql) # remove comment
        sql = sql.replace(",", ",\n") # break commas
        parsed = sqlparse.parse(sql) 
        root = parsed[0]
        pl.tokens.append(root)
        pl.value += row['QUERY_TEXT'] + "\n"
    
    return pl

#------------
# Helper 02 (used in Main 01)
#------------
def sql2tree(node, pnode_id='', local_node_id = 0, df = pd.DataFrame()):
    '''
    node is pl (parsed list of sqlparse objects)
    '''
    if node is None:
        return
    
    node_id = 'N0' if pnode_id=='' else f'{pnode_id}.{local_node_id}'
    ntype = str(node.ttype)
    Lines = len(str(node).splitlines())
    value = '  ' * node_id.count('.') + str(node.value) # add indentation
    
    new_row = pd.Series({'Node_ID': node_id, 'Type': ntype, 'Lines': Lines, 'Value': value,})

    df = df.append(new_row, ignore_index=True)
    
    if node.ttype is None :
        for i, child in enumerate(node.tokens):
            df = sql2tree(child, node_id, i, df) # index start from 0
            
    return df

#------------
# Helper 03A (used by Main 01)
#------------

def get_from_nodes(node, pnode_id='', local_node_id=0, from_nodes=[], parent_has_from=True):
    '''
    node is pl (parsed list of sqlparse objects)
    '''
    if node is None:
        return from_nodes

    node_id = 'N0' if pnode_id == '' else f'{pnode_id}.{local_node_id}'
    
    from_index = None
    has_from = False  # Initialize has_from as False
            
    if node.ttype is None:
        # add this node
        if node_id != 'N0': # N0 has special problem, exclude from list

            from_nodes.append(node_id)
        
        # get into a subset of children nodes 
        for f, cnode in enumerate(node.tokens):
            if cnode.value.lower() == 'from':  # Compare case-insensitive
                from_index = f
                has_from = True
        
        for i, cnode in enumerate(node.tokens):
            if ((from_index is not None and i > from_index) or (from_index is None and parent_has_from)) and len(cnode.value)>250:
                from_nodes = get_from_nodes(cnode, node_id, i, from_nodes, has_from)           

    return from_nodes
        
#------------
# Helper 03B (used by Main 01)
#------------
def get_nodes_with_text(df, include_list):
    
    cond1 = df['Node_ID'] != 'N0' # remove top level, which is a extra level to handle multiple statements
    cond2 = df['Value'].str.contains('|'.join(include_list)) # contains keywords
    cond3 = df['Type']=='None' # only plot nodes with children
    cond4 = df['Value'].str.len()>0 # 200

    result = df[cond1 & cond2 & cond3 & cond4]['Node_ID'].to_list()
    return result
    

#------------
# Helper 04 (used in 06 gen_nodes)
#------------
def getNodebyID(s, node_id):
        
    ids = node_id.split('.')
    s = s.tokens # handle first 0
    ids = ids[1:]
    for i in ids:
        s = s[int(i)]
            
    return s
    
#------------
# Helper 05 (used in 06 gen_nodes)
#------------
def comb_item(my_list, word):
    '''
    example: combine two nodes 'and ', 'var' into one node 'and var'
    '''
    new_list = []

    i = 0
    while i < len(my_list):
        if my_list[i] == word and i < len(my_list) - 1:
            new_string = f'{my_list[i]} {my_list[i + 1]}'
            new_list.append(new_string)
            i += 2
        else:
            new_list.append(my_list[i])
            i += 1
    return new_list

#------------
# Helper 06 (to provide cnodes list to Helper 07 find_nodes_between, called in 98 gen_nodes_list)
#------------
def get_cnodes_list(pl,node_id):
    '''
    pl: parsed list of sql statements
    '''

    pnode = getNodebyID(pl,node_id)
    
    cnodes = []

    for i, n in enumerate(pnode.tokens):
        cnode = {}
        cnode['index'] = str(i)
        cnode['text'] = str(n.value)
        cnodes.append(cnode)
    
    return cnodes

#------------
# Helper 07 (generate ignore in Helper 97 gen_nodes, called in 98 gen_nodes_list)
#------------
def find_nodes_between(pnode_id, nodes, start_text, end_text):
    '''
    get ['3','4'] etc. list of nodes to ignore in gen_nodes
    nodes: xx
    '''
    start_index = None
    end_index = None

    for i, node in enumerate(nodes):
        node['node_id'] = f'{pnode_id}.{node["index"]}'
        if node["text"] == start_text:
            start_index = i
        elif node["text"] == end_text:
            end_index = i
            break  # Stop searching once end_text is found

    if start_index is not None and end_index is not None and start_index < end_index:
        return [node['node_id'] for node in nodes[start_index + 1:end_index]]
    else:
        return []

#------------
# Helper 08 (called in Helper 98 gen_nodes_list)
#------------
def excl_nodes(source_nodes, exclude_list):
    
    def str_contains_substrs(input_string, substring_list):
        for substring in substring_list:
            if substring in input_string:
                return True
        return False
    
    return [node for node in source_nodes if not str_contains_substrs(node['text'], exclude_list)]

#------------
# Helper 09 (generate replace in Helper 97 gen_nodes, called in 98 gen_nodes_list)
#------------
def find_nodes_replace(pnode_id, cnodes, replace_nodes):
    replace_list = []
    
    for cnode in cnodes:
        if f"{pnode_id}.{cnode['index']}" in replace_nodes:
            replace_list.append(cnode['index'])
    
    return replace_list


#------------
# Helper 10 (called in Main 01)
#------------
def excl_steps(input_nodes, exclude_nodes):
    '''
    If any input node_id contains a string in the exclude_nodes list, then exclude it.
    '''
    remaining_nodes = []
    for node_id in input_nodes:
        exclude = False
        for enode_id in exclude_nodes:
            if enode_id in node_id:
                exclude = True
                break
        if not exclude:
            remaining_nodes.append(node_id)
    return remaining_nodes

#------------
# Helper 97 (used in 98 gen_nodes_list)
#------------
def gen_nodes(pl, node_id, replace=[], ignore=[], comb_word='', colors=[]):
    '''
    pl: root statment
    node_id: N0.1.1 etc.
    replace: ['1','2'] etc. list of child ids to ignore, replace node text with 'N0.1.1' etc.
    ignore: ['1','2'] etc. list of child ids to ignore, replace with '...'
    comb_word: e.g. combine two nodes: "and ", "a>b", into one node "and a>b"
    colors: ['white','red'] etc.
    '''

    # pnode label
    pnode_ = f'{node_id}'
    
    # cnodes labels
    pnode = getNodebyID(pl,node_id) 

    cnodes_ = []
    colors_ = []
    
    last_i = 0
    
    for i, n in enumerate(pnode.tokens):
        
        if str(n.ttype) not in [
            'Token.Text.Whitespace.Newline',
            'Token.Punctuation',
            'Token.Text.Whitespace']:
            
            # start with useful nodes: e.g. '1','4','7','9'
            if f'{i}' in replace:
                value = f'{node_id}.{i}' 
                cnodes_.append(value)
                if colors != []:
                    colors_.append(colors[i]) 
                
            elif (f'{i}' in ignore): # 4 is in ignore
                
                if (f'{last_i}' not in ignore): # 1 is not in ignore
                    value = '...'
                    cnodes_.append(value)
                    if colors != []:
                        colors_.append(colors[i])
                
            else: 
                value = n.value if len(n.value) <= 200 else n.value[:100] + '\n...\n' + n.value[-100:]
                cnodes_.append(value)
                if colors != []:
                    colors_.append(colors[i])
            
            last_i = i
                
    if comb_word != '':
        cnodes_ = comb_item(cnodes_, comb_word) # to deal with color later
    
    nodes = {} # dict for nodes
    nodes['pnode'] = pnode_ # pnode id, e.g. 'N0.1'
    nodes['cnodes'] = cnodes_ # list of cnodes text
    nodes['colors'] = colors_ # list of colors for each node
    return nodes

#------------
# Helper 98 (used in Main 01, by 99 sql_graph_list)
#------------
def gen_nodes_list(pl, include_list=[], addl_replace_list=[], attrs=[]):
    '''
    Generate pnode-cnodes sets for graph
    
    pl: statement object
    input_list: 'input list' from 03 get_nodes_with_text, a list of pnodes, ['N0.1', 'N0.1.1'] etc.
    addl_replact_list: ['N0.1', 'N0.1.1'] etc.
    attrs: list of var names to focus on
    
    return list of node lists: 
        each node list is generated by helper 06 gen_nodes
            nodes['pnode'] = pnode_ # pnode id, e.g. 'N0.1'
            nodes['cnodes'] = cnodes_ # list of cnodes text
            nodes['colors'] = colors # list of colors for each node
    '''

    output_list = []
    sf_nodes = []

    for node_id in include_list: #input_list from (03 get nodes by text)
                    
        cnodes = get_cnodes_list(pl,node_id) # helper 06, cnode['index'] = str(i), cnode['text'] = str(n.value)
        
        # REPLACE: if cnode in include_list or exclude_list, replace with node_id
        replace_list = find_nodes_replace(node_id, cnodes, include_list+addl_replace_list) # helper 09

        # IGNORE: LOGIC RE WHAT TO IGNORE, within "var node", all cnodes between select and from, except attrs to focus
        sf_nodes.extend(find_nodes_between(node_id, cnodes, 'select', 'from')) # helper 07, list of node_id.
        
        ignore_list = []
        if node_id in sf_nodes: # usually all select attrs are in one sf_node
            ignore_cnodes = excl_nodes(cnodes, attrs) # helper 08, return list of nodes obj (index, text)
            ignore_list = [str(cnode["index"]) for cnode in ignore_cnodes]
            
        # COLOR:
        color_list = []
        def has_attrs(attrs, node_text): # check if cnode text has any attr in attrs list
            for attr in attrs:
                if attr in node_text:
                    return True
            return False
            
        if attrs != []:
            for cnode in cnodes:
                if has_attrs(attrs, cnode['text']):
                    color = '#FFAAAA'
                elif cnode['text'][:3]=='N0.':
                    color = 'lightgrey'
                else:
                    color = 'white'                
                color_list.append(color)
        else: # no attr then highlight population
            color_list = []

        # helper 97 gen_nodes
        nodes = gen_nodes(pl,
                          node_id,
                          replace=replace_list, 
                          ignore=ignore_list, 
                          comb_word='', 
                          colors=color_list,)
        output_list.append(nodes)

    return output_list

#------------
# Helper 99 (used in Main)
#------------
def sql_graph_list(fname, nodes_list):
    '''
    plot a list of graphs
    fname: file name
    nodes_list: list of nodes from help 98 gen_nodes_list
    '''
    
    # create main graph
    dot = graphviz.Digraph()
    
    # add subgraphs
    for n, nodes in enumerate(nodes_list):

        # parse inputs
        pnode = nodes['pnode'] # pnode id, e.g. 'N0.1'
        cnodes = nodes['cnodes']
        colors = nodes['colors']
        
        # create subgraph
        sub = graphviz.Digraph(name=f'cluster_{pnode}') # cannot use pnode, otherwise no sub label
        
        # Set the label for the subgraph, list the step ids, start from 1
        sub.attr(label=f'Step {n+1}: {pnode}')

        # Add nodes
        sub.attr('node', shape='box')
        for i, node in enumerate(cnodes):
            if colors != []:
                sub.node(f'{pnode}.{i}', label=node, style='filled', fillcolor = colors[i])
            else: # quick hack, can get back to lightgrey
                temp = ['N0.', 'whe', 'tba']
                sub.node(f'{pnode}.{i}', label=node, style='filled', fillcolor='#FFAAAA' if (node[:3] in temp or node[:1]=='(') else 'white')

        # Add edges between nodes
        for i in range(len(cnodes)-1):
            sub.edge(f'{pnode}.{i}', f'{pnode}.{i+1}', constraint='true')
            
        # Add the subgraph to the main graph
        dot.subgraph(sub)

    # Render the graph
    dot.render(fname, format='svg', view=True) #png
    
    return dot

#----------
# Main 01
#----------
def audit_graph(fname, query, attrs=[], include_nodes=[], exclude_nodes=[],replace_nodes=[]):
    '''
    Args:
        query: A string that specifies the query.
        focus: pop or var
        include_nodes: A list of strings that specifies the nodes to include in the results.
        exclude_nodes: A list of strings that specifies the nodes to exclude from the results.
        replace_nodes: by default, if exclude, node will be flatten and show text.. 
                       unless in replace_nodes
    Returns:
        audit_graph.
    '''
    
    # ----------------
    # 01. Get SQL_Tree
    # ----------------
    
    pl = gen_pl(query)
    print(f'01. pl: {pl}')

    # Check if the file exists
    # TODO: handle sql_t
    if os.path.exists('sql_bt_dl.xlsx'):
        # If it exists, read it into a DataFrame
        df = pd.read_excel('sql_bt_dl.xlsx')
        print(f'01. File sql_bt_dl.xlsx loaded into df.')
    else:
        # If the file doesn't exist, run the sql2tree function
        df = sql2tree(pl)
        print(f'01. df = sql2tree(pl): done')
        # Save the DataFrame to an Excel file
        df.to_excel('sql_bt_dl.xlsx', index=False)
    
    # ----------------
    # 02. Get critical path
    # ----------------
    
    from_nodes = get_from_nodes(pl) # helper 03A, not working for test
    print(f'02. from nodes: {from_nodes}')

    attrs_nodes = get_nodes_with_text(df,attrs) # helper 03B
    print(f'02. attrs_nodes: {attrs_nodes}')
    
    critical_nodes = sorted(list(set(from_nodes + attrs_nodes)))
    print(f'02. critical_nodes: {critical_nodes}')
    
    post_excl_nodes = excl_steps(critical_nodes, exclude_nodes) # helper 10
    print(f'02. post_excl_nodes: {post_excl_nodes}')
    
    # ----------------
    # 03. Get audit graph for all steps on critical path
    # ----------------
    
    nodes_list = gen_nodes_list(pl,post_excl_nodes,replace_nodes,attrs) # helper 98
    
    g = sql_graph_list(fname, nodes_list) # helper 99
    
    return g

#-------------
# Call Main 01
#-------------
sql_t = '''
select 
var1, 
abs(var7) as var2, 
var3 
from 
(select var4, var5, var6 from source)
'''


'''g = audit_graph(fname = 'Test',
                query = sql_t,
                attrs = ['var2'],
                exclude_nodes = [],) # 'N0.0.4.5'
'''

"g = audit_graph(fname = 'Test',\n                query = sql_t,\n                attrs = ['var2'],\n                exclude_nodes = [],) # 'N0.0.4.5'\n"

In [29]:
var = 'Prin_Bal_at_Correct_Repay_Start_Dt'

g = audit_graph(fname = f'BT_LMR_{var}',
                query = sql_bt,
                attrs = [var],
                exclude_nodes = ['N0.0.7.10.7.0'], # do not show as a node in diagram 'N0.0.7.4'
                replace_nodes = ['N0.0.7.4'],
               )

01. pl: <__main__.MyStmts object at 0x7fd307727100>
01. File sql_bt_dl.xlsx loaded into df.
02. from nodes: ['N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7']
02. attrs_nodes: ['N0.0', 'N0.0.7', 'N0.0.7.4', 'N0.0.7.4.62', 'N0.0.7.4.62.4']
02. critical_nodes: ['N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0.7.4', 'N0.0.7.4.62', 'N0.0.7.4.62.4']
02. post_excl_nodes: ['N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0.7.4', 'N0.0.7.4.62', 'N0.0.7.4.62.4']


In [30]:
var = 'International_Student_Ind'

g = audit_graph(fname = f'BT_LMR_{var}',
                query = sql_bt,
                attrs = [var],
                exclude_nodes = ['N0.0.7.10.7.0'], # do not show as a node in diagram 'N0.0.7.4'
                replace_nodes = ['N0.0.7.4'],
               )

01. pl: <__main__.MyStmts object at 0x7fd30906d730>
01. File sql_bt_dl.xlsx loaded into df.
02. from nodes: ['N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7']
02. attrs_nodes: ['N0.0', 'N0.0.7', 'N0.0.7.4', 'N0.0.7.4.67', 'N0.0.7.4.67.4']
02. critical_nodes: ['N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0.7.4', 'N0.0.7.4.67', 'N0.0.7.4.67.4']
02. post_excl_nodes: ['N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0.7.4', 'N0.0.7.4.67', 'N0.0.7.4.67.4']


In [32]:
import pandas as pd

def read_column_from_excel(file_path, sheet_name, column_name):
    # Read the Excel file
    df = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
    
    # Extract the column data into a list
    column_data = df[column_name].tolist()
    
    return column_data

# Test the function
file_path = 'FW_ Sample Data, BT Calc vs. SF Challenger Query./BT_Calc_Data_CIRT256_20221018_LMR.xlsx'
sheet_name = 'Field Explanation - LMR'  # Change to your sheet name if different
column_name = 'Field'  # Change to the desired column name
fields = read_column_from_excel(file_path, sheet_name, column_name)
print(fields)

for f in fields:
    g = audit_graph(fname = f'BT_LMR_{f}',
                query = sql_bt,
                attrs = [f],
                exclude_nodes = ['N0.0.7.10.7.0'], # do not show as a node in diagram 'N0.0.7.4'
                replace_nodes = ['N0.0.7.4'],
               )


['UNIQUE_KEY', 'FORACID', 'ACID', 'Borrower_CIF', 'Cosigner_CIF', 'Current_Ln_Status', 'Correct_Repay_Start_Dt', 'Incorrect_Repay_Start_Dt', 'Impact_Start_Dt', 'Ln_Status_at_Correct_Repay_Start_Dt', 'Dt_of_Last_Interest_Accrual', 'Interest_Accrued_During_Period', 'Highest_Interest_Rate', 'Prin_Bal_at_Correct_Repay_Start_Dt', 'International_Student_Ind', 'Ln_Orig_from', 'TED_Dt', 'Run_Dt', 'Current_Ln_Bal']
01. pl: <__main__.MyStmts object at 0x7fd308fe7340>
01. File sql_bt_dl.xlsx loaded into df.
02. from nodes: ['N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7']
02. attrs_nodes: ['N0.0', 'N0.0.7', 'N0.0.7.4', 'N0.0.7.4.0', 'N0.0.7.4.0.4']
02. critical_nodes: ['N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0.7.4', 'N0.0.7.4.0', 'N0.0.7.4.0.4']
02. post_excl_nodes: ['N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.

01. pl: <__main__.MyStmts object at 0x7fd309037160>
01. File sql_bt_dl.xlsx loaded into df.
02. from nodes: ['N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7']
02. attrs_nodes: ['N0.0', 'N0.0.7', 'N0.0.7.4', 'N0.0.7.4.47', 'N0.0.7.4.47.4']
02. critical_nodes: ['N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0.7.4', 'N0.0.7.4.47', 'N0.0.7.4.47.4']
02. post_e

01. pl: <__main__.MyStmts object at 0x7fd30836ac70>
01. File sql_bt_dl.xlsx loaded into df.
02. from nodes: ['N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7', 'N0.0.7.10', 'N0.0.7.10.7', 'N0.0', 'N0.0.7',

In [ ]:
# TODO: debug no attr.. maybe add input for user to force red nodes
g = audit_graph(fname = 'BT_LMR',
                query = sql_bt,
                attrs = [],
                exclude_nodes = ['N0.0.7.10.7.0','N0.0.7.4'], # do not show as a node in diagram 'N0.0.7.4'
                replace_nodes = ['N0.0.7.4'],
               )

In [ ]:
#BACKUP 

g = audit_graph(fname = 'BT_LMR',
                query = sql_bt,
                attrs = ['Prin_Bal_at_Correct_Repay_Start_Dt'],
                exclude_nodes = ['N0.0.7.10.7.0',], # 'N0.0.7.4'
                replace_nodes = [])

#------------
# Helper 03 (used by Main 01)
#------------
def get_nodes_with_text(df, include_list):
    
    cond1 = df['Node_ID'] != 'N0' # remove top level, which is a extra level to handle multiple statements
    cond2 = df['Value'].str.contains('|'.join(include_list)) # contains keywords
    cond3 = df['Type']=='None' # only plot nodes with children
    
    result = df[cond1 & cond2 & cond3]['Node_ID'].to_list()
    
    return result # list of node_ids (input_list) for 07 nodes_list

In [2]:
# 00. read in sql string (see bottom)
# 01. gen sql_tree

import sqlparse
import pandas as pd
import re

# 01. gen sql_tree
def sql2tree(node, pnode_id='', local_node_id = 0, df = pd.DataFrame()):
    if node is None:
        return
    
    node_id = 'N0' if pnode_id=='' else f'{pnode_id}.{local_node_id}'
    ntype = str(node.ttype)
    Lines = len(str(node).splitlines())
    value = '  ' * node_id.count('.') + str(node.value) # add indentation
    
    new_row = pd.Series({'Node_ID': node_id, 'Type': ntype, 'Lines': Lines, 'Value': value,})

    df = df.append(new_row, ignore_index=True)
    
    if node.ttype is None :
        for i, child in enumerate(node.tokens):
            df = sql2tree(child, node_id, i, df) # index start from 0
            
    return df

class MyStmts:
    def __init__(self):
        self.tokens = []
        self.ttype = None
        self.value = ''

df_q = pd.DataFrame()

# USE sql_bt INPUT QUERY

df_q.at[0, 'QUERY_TEXT'] = sql_bt # assign sql text to df cell
pl = MyStmts()

for index, row in df_q.iterrows(): #multiple queries in SF
    sql = row['QUERY_TEXT']
    sql = re.sub(r'--.*', '', sql) # remove comment
    sql = sql.replace(",", ",\n") # break commas
    parsed = sqlparse.parse(sql) 
    root = parsed[0]
    pl.tokens.append(root)
    pl.value += row['QUERY_TEXT'] + "\n"
    
df = sql2tree(pl)
    
import xlwings as xw
wb = xw.Book('bt_lmr_sql_tree.xlsx')
sh = wb.sheets[0]
sh.clear_contents()
sh.range('A1').value = df
df

,Node_ID,Type,Lines,Value
0,N0,None,1.0,\nselect * from (\nselect gam.acid||to_char(st...
1,N0.0,None,596.0,\nselect * from (\nselect gam.acid||to_char(...
2,N0.0.0,Token.Text.Whitespace.Newline,1.0,\n
3,N0.0.1,Token.Keyword.DML,1.0,select
4,N0.0.2,Token.Text.Whitespace,1.0,
...,...,...,...,...
5838,N0.0.7.12.6.2,None,1.0,lam.acid
5839,N0.0.7.12.6.2.0,Token.Name,1.0,lam
5840,N0.0.7.12.6.2.1,Token.Punctuation,1.0,.
5841,N0.0.7.12.6.2.2,Token.Name,1.0,acid


In [3]:
dff = df[~df['Type'].isin([
    'Token.Text.Whitespace.Newline',
    'Token.Punctuation',
    'Token.Text.Whitespace'])].set_index('Node_ID')

import xlwings as xw
wb = xw.Book('bt_lmr_sql_tree_DL.xlsm')
sh = wb.sheets[0]
sh.clear_contents()
sh.range('A1').value = dff

dff

,Type,Lines,Value
Node_ID,,,
N0,None,1.0,\nselect * from (\nselect gam.acid||to_char(st...
N0.0,None,596.0,\nselect * from (\nselect gam.acid||to_char(...
N0.0.1,Token.Keyword.DML,1.0,select
N0.0.3,Token.Wildcard,1.0,*
N0.0.5,Token.Keyword,1.0,from
...,...,...,...
N0.0.7.12.6.0.2,Token.Name,1.0,acid
N0.0.7.12.6.1,Token.Operator.Comparison,1.0,=
N0.0.7.12.6.2,None,1.0,lam.acid


In [21]:
import graphviz

def getNodebyID(s, node_id):
    ids = node_id.split('.')
    s = s.tokens # handle first 0
    ids = ids[1:]
    for i in ids:
        s = s[int(i)]
    return s

def comb_item(my_list, word):
    new_list = []

    i = 0
    while i < len(my_list):
        if my_list[i] == word and i < len(my_list) - 1:
            new_string = f'{my_list[i]} {my_list[i + 1]}'
            new_list.append(new_string)
            i += 2
        else:
            new_list.append(my_list[i])
            i += 1
    return new_list

def gen_nodes(s, node_id, replace=[], ignore=[], comb_word='', colors=[]):

    # pnode label
    pnode_ = f'N{node_id}'
    
    # cnodes labels
    pnode = getNodebyID(pl,node_id)
    cnodes_ = []
    
    for i, n in enumerate(pnode.tokens):
        
        if str(n.ttype) not in [
            'Token.Text.Whitespace.Newline',
            'Token.Punctuation',
            'Token.Text.Whitespace']:

            if f'{i}' in replace:
                value = f'N{node_id}.{i}' 
                cnodes_.append(value)
                
            elif (f'{i}' in ignore):
                if (f'{i-1}' not in ignore):
                    value = '...'
                    cnodes_.append(value)
                
            else: 
                value = n.value
                cnodes_.append(value)
                
    if comb_word != '':
        cnodes_ = comb_item(cnodes_, comb_word)
    
    nodes = {} # dict for nodes
    nodes['pnode'] = pnode_
    nodes['cnodes'] = cnodes_
    nodes['colors'] = colors
    return nodes


def sql_graph_list(fname, nodes_list):
    
    # create main graph
    dot = graphviz.Digraph()
    
    # add subgraphs
    for nodes in nodes_list:

        # parse inputs
        pnode = nodes['pnode']
        cnodes = nodes['cnodes']
        colors = nodes['colors']
        
        # create subgraph
        sub = graphviz.Digraph(name=f'cluster_{pnode}') # cannot use pnode, otherwise no sub label
        
        # Set the label for the subgraph
        sub.attr(label=pnode)

        # Add nodes
        sub.attr('node', shape='box')
        for i, node in enumerate(cnodes):
            if colors != []:
                sub.node(f'{pnode}.{i}', label=node, style='filled', fillcolor = colors[i])
            else:
                sub.node(f'{pnode}.{i}', label=node, style='filled', fillcolor='lightgrey' if node[:3]=='N0.' else 'white')

        # Add edges between nodes
        for i in range(len(cnodes)-1):
            sub.edge(f'{pnode}.{i}', f'{pnode}.{i+1}', constraint='true')
            
        # Add the subgraph to the main graph
        dot.subgraph(sub)

    # Render the graph
    dot.render(fname, format='png', view=True)

nodes_list = [
    gen_nodes(pl,'0.0',['7']),
    gen_nodes(pl,'0.0.7',['4','10']),
    gen_nodes(pl,'0.0.7.4',['62'],[f'{i}' for i in list(range(0,62))+list(range(67,86))]),
    gen_nodes(pl,'0.0.7.4.62',['0'],[],'',['lightgrey']+['white']+['#FFAAAA']),
    gen_nodes(pl,'0.0.7.4.62.0',[],[],'',['white','#FFAAAA']*2+['white']),
    gen_nodes(pl,'0.0.7.10',['7'],),
    
]
sql_graph_list('BT_LMR', nodes_list)

In [ ]:
# backup
    gen_nodes(pl,'0.0.7',['0']),
    gen_nodes(pl,'0.0.7.0',['7']),
    gen_nodes(pl,'0.0.7.0.7',['61'],[f'{i}' for i in list(range(4,57))]),
    gen_nodes(pl,'0.0.7.0.7.61',['0']),
    gen_nodes(pl,'0.0.7.0.7.61.0',['12'],['5']),
    gen_nodes(pl,'0.0.7.0.7.61.0.12',['0'],),
    gen_nodes(pl,'0.0.7.0.7.61.0.12.0',['11','13'],[f'{i}' for i in list(range(4,7))]),
    gen_nodes(pl,'0.0.7.0.7.61.0.12.0.11',['10'],),
    gen_nodes(pl,'0.0.7.0.7.61.0.12.0.13',[],[],'and', ['white']*4+['#FFAAAA']*4+['white']*2+['#FFAAAA']*2),

In [2]:
sql_bt = '''
select * from (
select gam.acid||to_char(start_date,'MMDDYYYY')||to_char(end_date,'MMDDYYYY') as "UNIQUE_KEY",
gam.foracid as "FORACID",gam.acid as "ACID",gam.cif_id as "Borrower_CIF",

(select nma_key_id from tbaadm.aas where acid=gam.acid and del_flg='N' and acct_poa_as_rec_type='C') as "Cosigner_CIF",

Decode(Loan_status,'S','In School','H','Holiday Period','G','Grace','R','Repayment','P','Paid Off','X','Closed','W','Charged Off','D','Deferment','F','Forbearance','T','Restructure','I','Disaster Relief','Q','Settlement Pending','K','Write Off','Z','Settled') as "Current_Ln_Status",

start_date as "Correct_Repay_Start_Dt",
--end_date as "Incorrect_Repay_Start_Dt",end_date as "Impact_Start_Dt",

(case when loan_status in ('P','Q','W','Z','K')
then
case when Loan_status in ('P') then least(t.end_date,(select payoff_value_date from tbaadm.port where acid=gam.acid))
else
least(t.end_date,(select min(chrg_off_date) from tbaadm.la_coht where acid=gam.acid and reversed_flg='N')) end
else t.end_date end) as "Incorrect_Repay_Start_Dt",

(case when loan_status in ('P','Q','W','Z','K')
then
case when Loan_status in ('P') then least(t.end_date,(select payoff_value_date from tbaadm.port where acid=gam.acid))
else
least(t.end_date,(select min(chrg_off_date) from tbaadm.la_coht where acid=gam.acid and reversed_flg='N')) end
else t.end_date end) as "Impact_Start_Dt",

Decode(dfscust.dfs_utilities.get_loan_status_post(gam.acid,start_date),'S','In School','H','Holiday Period','G','Grace','R','Repayment','P','Paid Off','X','Closed','W','Charged Off','D','Deferment','F','Forbearance','T','Restructure','I','Disaster Relief','Q','Settlement Pending','K','Write Off','Z','Settled') as "Ln_Status_at_Correct_Repay_Start_Dt",


case when Loan_status in ('P') then (select payoff_value_date from tbaadm.port where acid=gam.acid)
when Loan_status in ('W','K','Q','Z') then 
(select min(chrg_off_date) from tbaadm.la_coht where acid=gam.acid and reversed_flg='N')
else null end as "Dt_of_Last_Interest_Accrual",

dfscust.dfs_utilities.int_amt_calc(gam.acid,t.start_date,t.end_date) as "Interest_Accrued_During_Period",

(select max(full_rate) full_rate from tbaadm.idt where entity_id=gam.acid
and (idt.start_date between (case when loan_status in ('P','Q','W','Z','K')
then
case when Loan_status in ('P') then least(t.end_date,(select payoff_value_date-1 from tbaadm.port where acid=gam.acid))
else
least(t.end_date,(select min(chrg_off_date)-1 from tbaadm.la_coht where acid=gam.acid and reversed_flg='N')) end
else t.end_date end)
and (select db_stat_date from tbaadm.gct) or
((case when loan_status in ('P','Q','W','Z','K')
then
case when Loan_status in ('P') then least(t.end_date,(select payoff_value_date-1 from tbaadm.port where acid=gam.acid))
else
least(t.end_date,(select min(chrg_off_date)-1 from tbaadm.la_coht where acid=gam.acid and reversed_flg='N')) end
else t.end_date end) between idt.start_date and idt.end_date and
(select db_stat_date from tbaadm.gct) between idt.start_date and idt.end_date) or
idt.end_date between (case when loan_status in ('P','Q','W','Z','K')
then
case when Loan_status in ('P') then least(t.end_date,(select payoff_value_date-1 from tbaadm.port where acid=gam.acid))
else
least(t.end_date,(select min(chrg_off_date)-1 from tbaadm.la_coht where acid=gam.acid and reversed_flg='N')) end
else t.end_date end) and (select db_stat_date from tbaadm.gct))) as "Highest_Interest_Rate",

(select 
case when value_date_bal >=0 then 0
else abs(value_date_bal) end 
from tbaadm.eab where acid=gam.acid and t.start_date between eod_date and end_eod_date) as "Prin_Bal_at_Correct_Repay_Start_Dt",

case 
when (select count(*) from crmuser.taxdetails where orgkey=gam.cif_id and taxidtype in ('PROXY','ITIN','EIN')) >0
then 'Y'
when (select count(*) from crmuser.taxdetails where orgkey=gam.cif_id and taxidtype='SSN' and substr(taxid,1,1)='9') >0
then 'Y' else 'N' end as "International_Student_Ind",

case when (select to_char(cutover_date,'YYYY') from dfscust.csllaadt where acid=gam.acid) = '2017' then 'Diploma'
when (select to_char(cutover_date,'YYYY') from dfscust.csllaadt where acid=gam.acid) = '2018' then 'Great Lakes'
else 'Finacle' end as "Ln_Orig_from",

xfer_eff_date as "TED_Dt",
(select db_stat_date from tbaadm.gct) as "Run_Dt",
case when clr_bal_amt >=0 then 0 else abs(clr_bal_amt)+int_dmd_os end as "Current_Ln_Bal"

 from tbaadm.gam,tbaadm.lam,
(
select 'SM5494538' acid,to_date('16-Nov-2018','DD-MON-YYYY') start_date,to_date('11-May-2019','DD-MON-YYYY') end_date from dual union all
select 'SM5540633' acid,to_date('23-Jul-2020','DD-MON-YYYY') start_date,to_date('24-Jul-2020','DD-MON-YYYY') end_date from dual union all
select 'SM5540672' acid,to_date('23-Jul-2020','DD-MON-YYYY') start_date,to_date('24-Jul-2020','DD-MON-YYYY') end_date from dual union all
select 'SM7857199' acid,to_date('24-Dec-2019','DD-MON-YYYY') start_date,to_date('17-Jun-2020','DD-MON-YYYY') end_date from dual union all
select 'SM5701180' acid,to_date('18-Oct-2018','DD-MON-YYYY') start_date,to_date('04-May-2019','DD-MON-YYYY') end_date from dual union all
select 'SM5538786' acid,to_date('05-Nov-2020','DD-MON-YYYY') start_date,to_date('06-Nov-2020','DD-MON-YYYY') end_date from dual union all
select 'SM5621552' acid,to_date('03-Sep-2020','DD-MON-YYYY') start_date,to_date('04-Sep-2020','DD-MON-YYYY') end_date from dual union all
select 'SM5621521' acid,to_date('03-Sep-2020','DD-MON-YYYY') start_date,to_date('04-Sep-2020','DD-MON-YYYY') end_date from dual union all
select 'SM5612596' acid,to_date('18-Oct-2018','DD-MON-YYYY') start_date,to_date('15-Dec-2018','DD-MON-YYYY') end_date from dual union all
select 'SM5612630' acid,to_date('18-Oct-2018','DD-MON-YYYY') start_date,to_date('15-Dec-2018','DD-MON-YYYY') end_date from dual union all
select 'SM5526726' acid,to_date('07-Jul-2020','DD-MON-YYYY') start_date,to_date('22-Dec-2020','DD-MON-YYYY') end_date from dual union all
select 'SM5622143' acid,to_date('22-Mar-2018','DD-MON-YYYY') start_date,to_date('23-Sep-2019','DD-MON-YYYY') end_date from dual union all
select 'SM5495196' acid,to_date('16-Nov-2018','DD-MON-YYYY') start_date,to_date('10-Aug-2019','DD-MON-YYYY') end_date from dual union all
select 'SM5495157' acid,to_date('16-Nov-2018','DD-MON-YYYY') start_date,to_date('10-Aug-2019','DD-MON-YYYY') end_date from dual union all
select 'SM5504710' acid,to_date('15-Jul-2020','DD-MON-YYYY') start_date,to_date('17-Jul-2020','DD-MON-YYYY') end_date from dual union all
select 'SM5540533' acid,to_date('15-Jun-2020','DD-MON-YYYY') start_date,to_date('23-Jun-2020','DD-MON-YYYY') end_date from dual union all
select 'SM5624131' acid,to_date('16-Nov-2018','DD-MON-YYYY') start_date,to_date('21-Dec-2019','DD-MON-YYYY') end_date from dual union all
select 'SM5624169' acid,to_date('16-Nov-2018','DD-MON-YYYY') start_date,to_date('21-Dec-2019','DD-MON-YYYY') end_date from dual union all
select 'SM6287539' acid,to_date('13-Jan-2021','DD-MON-YYYY') start_date,to_date('15-Jan-2021','DD-MON-YYYY') end_date from dual union all
select 'SM5201433' acid,to_date('13-Jan-2021','DD-MON-YYYY') start_date,to_date('15-Jan-2021','DD-MON-YYYY') end_date from dual union all
select 'SM5238330' acid,to_date('26-May-2020','DD-MON-YYYY') start_date,to_date('27-Jun-2020','DD-MON-YYYY') end_date from dual union all
select 'SM5653801' acid,to_date('08-May-2021','DD-MON-YYYY') start_date,to_date('25-May-2021','DD-MON-YYYY') end_date from dual union all
select 'SM5653838' acid,to_date('08-May-2021','DD-MON-YYYY') start_date,to_date('25-May-2021','DD-MON-YYYY') end_date from dual union all
select 'SM5490220' acid,to_date('21-Nov-2018','DD-MON-YYYY') start_date,to_date('13-Nov-2022','DD-MON-YYYY') end_date from dual union all
select 'SM5494673' acid,to_date('06-Jun-2019','DD-MON-YYYY') start_date,to_date('07-Jun-2019','DD-MON-YYYY') end_date from dual union all
select 'SM5483928' acid,to_date('10-Oct-2018','DD-MON-YYYY') start_date,to_date('11-Jan-2019','DD-MON-YYYY') end_date from dual union all
select 'SM5483969' acid,to_date('10-Oct-2018','DD-MON-YYYY') start_date,to_date('11-Jan-2019','DD-MON-YYYY') end_date from dual union all
select 'SM5200178' acid,to_date('11-May-2021','DD-MON-YYYY') start_date,to_date('13-Nov-2022','DD-MON-YYYY') end_date from dual union all
select 'SM5671895' acid,to_date('28-Nov-2018','DD-MON-YYYY') start_date,to_date('18-May-2019','DD-MON-YYYY') end_date from dual union all
select 'SM5773779' acid,to_date('28-Nov-2018','DD-MON-YYYY') start_date,to_date('11-May-2020','DD-MON-YYYY') end_date from dual union all
select 'SM5794545' acid,to_date('14-Aug-2020','DD-MON-YYYY') start_date,to_date('17-Aug-2020','DD-MON-YYYY') end_date from dual union all
select 'SM5794582' acid,to_date('14-Aug-2020','DD-MON-YYYY') start_date,to_date('17-Aug-2020','DD-MON-YYYY') end_date from dual union all
select 'SM5820933' acid,to_date('08-Jan-2019','DD-MON-YYYY') start_date,to_date('18-May-2019','DD-MON-YYYY') end_date from dual union all
select 'SM5820971' acid,to_date('08-Jan-2019','DD-MON-YYYY') start_date,to_date('18-May-2019','DD-MON-YYYY') end_date from dual union all
select 'SM5778886' acid,to_date('26-Dec-2018','DD-MON-YYYY') start_date,to_date('07-Dec-2019','DD-MON-YYYY') end_date from dual union all
select 'SM5778923' acid,to_date('26-Dec-2018','DD-MON-YYYY') start_date,to_date('07-Dec-2019','DD-MON-YYYY') end_date from dual union all
select 'SM5440518' acid,to_date('15-Jul-2020','DD-MON-YYYY') start_date,to_date('17-Jul-2020','DD-MON-YYYY') end_date from dual union all
select 'SM5805656' acid,to_date('16-Jun-2020','DD-MON-YYYY') start_date,to_date('08-Jul-2020','DD-MON-YYYY') end_date from dual union all
select 'SM5805736' acid,to_date('16-Jun-2020','DD-MON-YYYY') start_date,to_date('08-Jul-2020','DD-MON-YYYY') end_date from dual union all
select 'SM5805695' acid,to_date('16-Jun-2020','DD-MON-YYYY') start_date,to_date('08-Jul-2020','DD-MON-YYYY') end_date from dual union all
select 'SM5805778' acid,to_date('16-Jun-2020','DD-MON-YYYY') start_date,to_date('08-Jul-2020','DD-MON-YYYY') end_date from dual union all
select 'SM5667576' acid,to_date('10-Jun-2020','DD-MON-YYYY') start_date,to_date('23-Jun-2020','DD-MON-YYYY') end_date from dual union all
select 'SM5667538' acid,to_date('10-Jun-2020','DD-MON-YYYY') start_date,to_date('23-Jun-2020','DD-MON-YYYY') end_date from dual union all
select 'SM5805303' acid,to_date('08-Jul-2020','DD-MON-YYYY') start_date,to_date('11-Dec-2020','DD-MON-YYYY') end_date from dual union all
select 'SM5805341' acid,to_date('08-Jul-2020','DD-MON-YYYY') start_date,to_date('11-Dec-2020','DD-MON-YYYY') end_date from dual union all
select 'SM5768190' acid,to_date('17-Oct-2018','DD-MON-YYYY') start_date,to_date('06-Mar-2019','DD-MON-YYYY') end_date from dual union all
select 'SM5697114' acid,to_date('23-May-2019','DD-MON-YYYY') start_date,to_date('24-Sep-2019','DD-MON-YYYY') end_date from dual union all
select 'SM4596599' acid,to_date('16-Aug-2019','DD-MON-YYYY') start_date,to_date('14-Dec-2019','DD-MON-YYYY') end_date from dual union all
select 'SM4596578' acid,to_date('16-Aug-2019','DD-MON-YYYY') start_date,to_date('14-Dec-2019','DD-MON-YYYY') end_date from dual union all
select 'SM4772295' acid,to_date('14-Nov-2017','DD-MON-YYYY') start_date,to_date('04-Dec-2017','DD-MON-YYYY') end_date from dual union all
select 'SM4514481' acid,to_date('01-Feb-2018','DD-MON-YYYY') start_date,to_date('08-Jun-2019','DD-MON-YYYY') end_date from dual union all
select 'SM4514427' acid,to_date('01-Feb-2018','DD-MON-YYYY') start_date,to_date('10-Jul-2021','DD-MON-YYYY') end_date from dual union all
select 'SM4514401' acid,to_date('01-Feb-2018','DD-MON-YYYY') start_date,to_date('10-Jul-2021','DD-MON-YYYY') end_date from dual union all
select 'SM4514454' acid,to_date('01-Feb-2018','DD-MON-YYYY') start_date,to_date('10-Jul-2021','DD-MON-YYYY') end_date from dual union all
select 'SM4531917' acid,to_date('19-Oct-2017','DD-MON-YYYY') start_date,to_date('01-Oct-2020','DD-MON-YYYY') end_date from dual union all
select 'SM4752166' acid,to_date('16-Nov-2018','DD-MON-YYYY') start_date,to_date('25-Apr-2019','DD-MON-YYYY') end_date from dual union all
select 'SM5866675' acid,to_date('16-Nov-2018','DD-MON-YYYY') start_date,to_date('09-Dec-2019','DD-MON-YYYY') end_date from dual union all
select 'SM4747729' acid,to_date('13-Apr-2018','DD-MON-YYYY') start_date,to_date('01-Jul-2019','DD-MON-YYYY') end_date from dual union all
select 'SM4424700' acid,to_date('27-Jul-2021','DD-MON-YYYY') start_date,to_date('26-Oct-2021','DD-MON-YYYY') end_date from dual union all
select 'SM4424742' acid,to_date('27-Jul-2021','DD-MON-YYYY') start_date,to_date('26-Oct-2021','DD-MON-YYYY') end_date from dual union all
select 'SM5865347' acid,to_date('22-Jul-2020','DD-MON-YYYY') start_date,to_date('30-Sep-2020','DD-MON-YYYY') end_date from dual union all
select 'SM5865353' acid,to_date('22-Jul-2020','DD-MON-YYYY') start_date,to_date('30-Sep-2020','DD-MON-YYYY') end_date from dual union all
select 'SM4563239' acid,to_date('22-Jul-2020','DD-MON-YYYY') start_date,to_date('30-Sep-2020','DD-MON-YYYY') end_date from dual union all
select 'SM4585936' acid,to_date('13-Nov-2017','DD-MON-YYYY') start_date,to_date('25-Apr-2018','DD-MON-YYYY') end_date from dual union all
select 'SM4585965' acid,to_date('13-Nov-2017','DD-MON-YYYY') start_date,to_date('23-Oct-2018','DD-MON-YYYY') end_date from dual union all
select 'SM4586290' acid,to_date('13-Nov-2017','DD-MON-YYYY') start_date,to_date('23-Oct-2018','DD-MON-YYYY') end_date from dual union all
select 'SM4586334' acid,to_date('13-Nov-2017','DD-MON-YYYY') start_date,to_date('23-Oct-2018','DD-MON-YYYY') end_date from dual union all
select 'SM4586357' acid,to_date('13-Nov-2017','DD-MON-YYYY') start_date,to_date('23-Oct-2018','DD-MON-YYYY') end_date from dual union all
select 'SM4586314' acid,to_date('13-Nov-2017','DD-MON-YYYY') start_date,to_date('23-Oct-2018','DD-MON-YYYY') end_date from dual union all
select 'SM4645260' acid,to_date('26-Oct-2017','DD-MON-YYYY') start_date,to_date('09-May-2022','DD-MON-YYYY') end_date from dual union all
select 'SM4691399' acid,to_date('19-Nov-2017','DD-MON-YYYY') start_date,to_date('09-Feb-2018','DD-MON-YYYY') end_date from dual union all
select 'SM4703819' acid,to_date('27-Feb-2018','DD-MON-YYYY') start_date,to_date('16-Mar-2018','DD-MON-YYYY') end_date from dual union all
select 'SM4703782' acid,to_date('27-Feb-2018','DD-MON-YYYY') start_date,to_date('16-Mar-2018','DD-MON-YYYY') end_date from dual union all
select 'SM4703848' acid,to_date('27-Feb-2018','DD-MON-YYYY') start_date,to_date('16-Mar-2018','DD-MON-YYYY') end_date from dual union all
select 'SM4758745' acid,to_date('09-Feb-2018','DD-MON-YYYY') start_date,to_date('02-Jul-2019','DD-MON-YYYY') end_date from dual union all
select 'SM4759102' acid,to_date('09-Feb-2018','DD-MON-YYYY') start_date,to_date('02-Jul-2019','DD-MON-YYYY') end_date from dual union all
select 'SM4758776' acid,to_date('09-Feb-2018','DD-MON-YYYY') start_date,to_date('02-Jul-2019','DD-MON-YYYY') end_date from dual union all
select 'SM4408557' acid,to_date('09-Jun-2019','DD-MON-YYYY') start_date,to_date('01-Aug-2019','DD-MON-YYYY') end_date from dual union all
select 'SM4412414' acid,to_date('31-Jul-2020','DD-MON-YYYY') start_date,to_date('29-Sep-2020','DD-MON-YYYY') end_date from dual
) t
where gam.acid=t.acid and gam.acid=lam.acid)'''